In [1]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from collections import Counter

In [2]:


'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
< naver 뉴스 전문 가져오기 >_select 사용
- 네이버 뉴스만 가져와서 결과값 조금 작음 
- 결과 메모장 저장 -> 엑셀로 저장 
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
RESULT_PATH = 'C:\\Users\\Kyungbok Lee\\Documents\\git\\python\\crawler&wordcloud\\'
now = datetime.now() #파일이름 현 시간으로 저장하기

def get_news(n_url):
    news_detail = []

    breq = requests.get(n_url)
    bsoup = BeautifulSoup(breq.content, 'html.parser')

    title = bsoup.select('h3#articleTitle')[0].text  #대괄호는  h3#articleTitle 인 것중 첫번째 그룹만 가져오겠다.
    news_detail.append(title)

    pdate = bsoup.select('.t11')[0].get_text()[:11]
    news_detail.append(pdate)

    _text = bsoup.select('#articleBodyContents')[0].get_text().replace('\n', " ")
    btext = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_detail.append(btext.strip())
  
    news_detail.append(n_url)
    
    pcompany = bsoup.select('#footer address')[0].a.get_text()
    news_detail.append(pcompany)

    return news_detail

def crawler(maxpage,query,s_date,e_date):

    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지
    f = open("C:\\Users\\Kyungbok Lee\\Documents\\git\\python\\crawler&wordcloud\\contents_text.txt", 'w', encoding='utf-8')
    
    while page < maxpage_t:
    
        print(page)
    
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=0&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        
        req = requests.get(url)
        print(url)
        cont = req.content
        soup = BeautifulSoup(cont, 'html.parser')
            #print(soup)
    
        for urls in soup.select("._sp_each_url"):
            try :
                #print(urls["href"])
                if urls["href"].startswith("https://news.naver.com"):
                    #print(urls["href"])
                    news_detail = get_news(urls["href"])
                        # pdate, pcompany, title, btext
                    f.write("{}\t{}\t{}\t{}\t{}\n".format(news_detail[1], news_detail[4], news_detail[0], news_detail[2],news_detail[3]))  # new style
            except Exception as e:
                print(e)
                continue
        page += 10
    
    
    f.close()
    
def excel_make():
    data = pd.read_csv(RESULT_PATH+'contents_text.txt', sep='\t',header=None, error_bad_lines=False)
    data.columns = ['years','company','title','contents','link']
    print(data)
    
    xlsx_outputFileName = '%s-%s-%s  %s시 %s분 %s초 result.xlsx' % (now.year, now.month, now.day, now.hour, now.minute, now.second)
    #xlsx_name = 'result' + '.xlsx'
    data.to_excel(RESULT_PATH+xlsx_outputFileName, encoding='utf-8')

#crawler(100,"다이슨",2019.01.01,2019.04.28)
    
def main():
    maxpage = input("최대 출력할 페이지수 입력하시오: ") 
    query = input("검색어 입력: ")
    s_date = input("시작날짜 입력(2019.01.01):")  #2019.01.01
    e_date = input("끝날짜 입력(2019.04.28):")   #2019.04.28
    crawler(maxpage,query,s_date,e_date) #검색된 네이버뉴스의 기사내용을 크롤링합니다. 
    
    excel_make() #엑셀로 만들기 
main()

최대 출력할 페이지수 입력하시오: 100
검색어 입력: e편한세상
시작날짜 입력(2019.01.01):2019.07.13
끝날짜 입력(2019.04.28):2020.07.13
1
https://search.naver.com/search.naver?where=news&query=e편한세상&sort=0&ds=2019.07.13&de=2020.07.13&nso=so%3Ar%2Cp%3Afrom20190713to20200713%2Ca%3A&start=1
11
https://search.naver.com/search.naver?where=news&query=e편한세상&sort=0&ds=2019.07.13&de=2020.07.13&nso=so%3Ar%2Cp%3Afrom20190713to20200713%2Ca%3A&start=11
21
https://search.naver.com/search.naver?where=news&query=e편한세상&sort=0&ds=2019.07.13&de=2020.07.13&nso=so%3Ar%2Cp%3Afrom20190713to20200713%2Ca%3A&start=21
31
https://search.naver.com/search.naver?where=news&query=e편한세상&sort=0&ds=2019.07.13&de=2020.07.13&nso=so%3Ar%2Cp%3Afrom20190713to20200713%2Ca%3A&start=31
41
https://search.naver.com/search.naver?where=news&query=e편한세상&sort=0&ds=2019.07.13&de=2020.07.13&nso=so%3Ar%2Cp%3Afrom20190713to20200713%2Ca%3A&start=41
51
https://search.naver.com/search.naver?where=news&query=e편한세상&sort=0&ds=2019.07.13&de=2020.07.13&nso=so%3Ar%2Cp%3Afrom20190713t

521
https://search.naver.com/search.naver?where=news&query=e편한세상&sort=0&ds=2019.07.13&de=2020.07.13&nso=so%3Ar%2Cp%3Afrom20190713to20200713%2Ca%3A&start=521
531
https://search.naver.com/search.naver?where=news&query=e편한세상&sort=0&ds=2019.07.13&de=2020.07.13&nso=so%3Ar%2Cp%3Afrom20190713to20200713%2Ca%3A&start=531
541
https://search.naver.com/search.naver?where=news&query=e편한세상&sort=0&ds=2019.07.13&de=2020.07.13&nso=so%3Ar%2Cp%3Afrom20190713to20200713%2Ca%3A&start=541
551
https://search.naver.com/search.naver?where=news&query=e편한세상&sort=0&ds=2019.07.13&de=2020.07.13&nso=so%3Ar%2Cp%3Afrom20190713to20200713%2Ca%3A&start=551
561
https://search.naver.com/search.naver?where=news&query=e편한세상&sort=0&ds=2019.07.13&de=2020.07.13&nso=so%3Ar%2Cp%3Afrom20190713to20200713%2Ca%3A&start=561
571
https://search.naver.com/search.naver?where=news&query=e편한세상&sort=0&ds=2019.07.13&de=2020.07.13&nso=so%3Ar%2Cp%3Afrom20190713to20200713%2Ca%3A&start=571
581
https://search.naver.com/search.naver?where=news&query

b'Skipping line 3: expected 5 fields, saw 8\nSkipping line 4: expected 5 fields, saw 8\nSkipping line 5: expected 5 fields, saw 8\nSkipping line 6: expected 5 fields, saw 8\nSkipping line 11: expected 5 fields, saw 8\nSkipping line 12: expected 5 fields, saw 8\nSkipping line 21: expected 5 fields, saw 8\nSkipping line 34: expected 5 fields, saw 8\nSkipping line 43: expected 5 fields, saw 8\nSkipping line 44: expected 5 fields, saw 8\nSkipping line 45: expected 5 fields, saw 8\nSkipping line 46: expected 5 fields, saw 8\nSkipping line 61: expected 5 fields, saw 8\nSkipping line 62: expected 5 fields, saw 8\nSkipping line 82: expected 5 fields, saw 8\nSkipping line 91: expected 5 fields, saw 8\nSkipping line 94: expected 5 fields, saw 8\nSkipping line 95: expected 5 fields, saw 8\nSkipping line 101: expected 5 fields, saw 8\nSkipping line 102: expected 5 fields, saw 8\nSkipping line 103: expected 5 fields, saw 8\nSkipping line 104: expected 5 fields, saw 8\nSkipping line 108: expected 5 

           years         company                                     title  \
0    2020.07.13.            데일리안   ‘서울보다 나은’ 인천 오피스텔 수익률…’e편한세상 시티 부평역’ 주목   
1    2020.07.11.             뉴스1       6·17 대책 후폭풍에도…'e편한세상 시티 부평역' 규제 비껴가   
2    2020.07.12.             머니S   '서울보다 나은' 인천 오피스텔 수익률…'e편한세상 시티 부평역' 주목   
3    2020.07.09.     inews24.com        '6·17대책'후폭풍…규제 비껴나간 'e편한세상 시티 부평역'   
4    2020.07.09.  financial news     '6·17대책' 후폭풍에도 규제 비껴나간 'e편한세상 시티 부평역'   
..           ...             ...                                       ...   
671  2019.10.31.          디지털타임스  “가장 살고 싶은 아파트 브랜드는?”…‘래미안·e편한세상·자이’ top3   
672  2020.05.22.         아시아경제신문        영천 첫 1군 브랜드 아파트 e편한세상 분양…1210가구 규모   
673  2019.07.24.           헤럴드경제      대림산업, ‘e편한세상 일산 어반스카이’ 오피스텔 8월 분양 예정   
674  2019.07.24.          디지털타임스    대림산업, `e편한세상 일산 어반스카이` 오피스텔 225실 8월 분양   
675  2019.07.24.         아시아경제신문        대림산업, 'e편한세상 일산 어반스카이' 오피스텔 다음달 분양   

                                              contents  \
0    

In [3]:
contents = ""
df = pd.read_excel("2020-7-13  23시 44분 3초 result.xlsx")

In [4]:

for i in df["contents"]:
    contents += i

In [5]:

 
wordcloud = WordCloud(font_path='font/NanumGothic.ttf', background_color='white',
                      width = 1000,
                     height = 1000,
                     max_words = 100,
                     max_font_size = 300).generate(contents)


In [6]:
titles = ""

In [7]:
for i in df["title"]:
    titles += i

In [8]:
from konlpy.tag import Twitter

In [9]:
twitter = Twitter()

C:\Users\Kyungbok Lee\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [10]:
title_tag = twitter.pos(titles)

In [11]:
title_tag

[('‘', 'Foreign'),
 ('서울', 'Noun'),
 ('보다', 'Josa'),
 ('나은', 'Noun'),
 ('’', 'Punctuation'),
 ('인천', 'Noun'),
 ('오피스텔', 'Noun'),
 ('수익률', 'Noun'),
 ('…’', 'Punctuation'),
 ('e', 'Alpha'),
 ('편', 'Noun'),
 ('한세상', 'Noun'),
 ('시티', 'Noun'),
 ('부평역', 'Noun'),
 ('’', 'Punctuation'),
 ('주목', 'Noun'),
 ('6', 'Number'),
 ('·', 'Punctuation'),
 ('17', 'Number'),
 ('대책', 'Noun'),
 ('후', 'Noun'),
 ('폭풍', 'Noun'),
 ('에도', 'Josa'),
 ("…'", 'Punctuation'),
 ('e', 'Alpha'),
 ('편', 'Noun'),
 ('한세상', 'Noun'),
 ('시티', 'Noun'),
 ('부평역', 'Noun'),
 ("'", 'Punctuation'),
 ('규제', 'Noun'),
 ('비', 'Noun'),
 ('껴가', 'Verb'),
 ("'", 'Punctuation'),
 ('서울', 'Noun'),
 ('보다', 'Josa'),
 ('나은', 'Noun'),
 ("'", 'Punctuation'),
 ('인천', 'Noun'),
 ('오피스텔', 'Noun'),
 ('수익률', 'Noun'),
 ("…'", 'Punctuation'),
 ('e', 'Alpha'),
 ('편', 'Noun'),
 ('한세상', 'Noun'),
 ('시티', 'Noun'),
 ('부평역', 'Noun'),
 ("'", 'Punctuation'),
 ('주목', 'Noun'),
 ("'", 'Punctuation'),
 ('6', 'Number'),
 ('·', 'Punctuation'),
 ('17', 'Number'),
 ('대책', '

In [12]:
# Title 형태분석 
title_noun_list = []
title_adjective_list =[]
for word, tag in title_tag:
    if tag in ["Noun"]:
        title_noun_list.append(word)
    elif tag in ["Adjective"]:
        title_adjective_list.append(word)
    

In [14]:
T_N_count

Counter({'서울': 2,
         '나은': 2,
         '인천': 2,
         '오피스텔': 74,
         '수익률': 2,
         '편': 677,
         '한세상': 677,
         '시티': 131,
         '부평역': 140,
         '주목': 27,
         '대책': 10,
         '후': 5,
         '폭풍': 4,
         '규제': 12,
         '비': 7,
         '권': 48,
         '미래': 6,
         '가치': 10,
         '눈길': 19,
         '창원': 4,
         '파크': 5,
         '센트럴': 37,
         '임차': 1,
         '모집': 2,
         '최대': 4,
         '거주': 1,
         '보장': 1,
         '대림': 272,
         '산업': 233,
         '평택': 10,
         '센터': 36,
         '포레': 61,
         '분양': 318,
         '내달': 53,
         '먼저': 1,
         '살': 2,
         '보고': 1,
         '결정': 1,
         '주거': 1,
         '라이프': 5,
         '주택': 23,
         '전시관': 26,
         '개관': 30,
         '간': 1,
         '복선전철': 2,
         '수혜': 8,
         '시장': 2,
         '영천': 7,
         '수요': 14,
         '브랜드': 70,
         '리뉴얼': 47,
         '첫': 17,
         '단지': 34,
       

In [15]:
wc = WordCloud(font_path='font/NanumGothic.ttf', background_color='white',
                     width = 1000,
                     height = 1000,
                     max_words = 100,
                     max_font_size = 300)


In [16]:
# title adjective
wc.generate_from_frequencies(dict(T_A_count))
wc.to_file("Title_adjective2.png")

In [17]:
# title noun
wc.generate_from_frequencies(dict(T_N_count))
wc.to_file("Title_noun2.png")

In [18]:
### content 형태분석 
content_noun_list = []
content_adjective_list =[]
for word, tag in title_tag:
    if tag in ["Noun"]:
        content_noun_list.append(word)
    elif tag in ["Adjective"]:
        content_adjective_list.append(word)
    

In [19]:
C_N_count = Counter(title_noun_list)
C_A_count = Counter(title_adjective_list)

In [20]:
# Content adjective
wc.generate_from_frequencies(dict(C_N_count))
wc.to_file("Content_noun2.png")

In [21]:
# Content adjective
wc.generate_from_frequencies(dict(C_A_count))
wc.to_file("Content_adjective2.png")

In [ ]:
# TOP 10 
wc = WordCloud(font_path='font/NanumGothic.ttf', background_color='white',
                     width = 1000,
                     height = 1000,
                     max_words = 100,
                     max_font_size = 300)
wc.g
array = wc.to_array()
fig = plt.figure(figsize=(10, 10))
plt.imshow(array, interpolation="bilinear")
plt.axis('off')
plt.show()